In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, accuracy_score, recall_score, balanced_accuracy_score, roc_curve
from sklearn.utils import shuffle
from sklearn.datasets import make_classification
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder

from imblearn.over_sampling import SMOTE

# Reading, Preprocessing and EDA

In [3]:
Train_path = "../datasets/adult/adult.csv"
# Test_path = "../datasets/adult/adult.csv"
train_df = pd.read_csv(Train_path, header = 0)
# test_df = pd.read_csv(Test_path, header = None)
print("done")

done


In [4]:
# train_df = train_df.iloc[1: , :]
# test_df = test_df.iloc[1:, :]
# train_df, test_df = train_test_split(train_df, train_size = 0.9, random_state = 42) 

In [5]:
print(train_df.isnull().sum().sum(), train_df.isna().sum().sum())
# print(test_df.isnull().sum().sum(), train_df.isna().sum().sum())

0 0


In [6]:
list_dtype = [train_df[i].dtype for i in train_df.columns]
print(dict((i, list_dtype.count(i)) for i in list_dtype))
# list_dtype = [test_df[i].dtype for i in test_df.columns]
# print(dict((i, list_dtype.count(i)) for i in list_dtype))

{dtype('int64'): 6, dtype('O'): 9}


In [7]:
train_df = train_df.replace('?', np.NaN)
# test_df = test_df.replace('?', np.NaN)

In [8]:
print(train_df.isna().sum().sum())
# print(test_df.isna().sum().sum())

6465


In [9]:
train_df[:5]

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K


In [10]:
num_col = ['age', 'fnlwgt', 'educational-num', 'capital-gain', 'capital-loss', 'hours-per-week']

In [11]:
list_dtype = [train_df[i].dtype for i in train_df.columns]
print(dict((i, list_dtype.count(i)) for i in list_dtype))
# list_dtype = [test_df[i].dtype for i in test_df.columns]
# print(dict((i, list_dtype.count(i)) for i in list_dtype))

{dtype('int64'): 6, dtype('O'): 9}


In [12]:
# num_col = {0, 2, 4, 10, 11, 12}
for col in num_col:
    train_df[col] = pd.to_numeric(train_df[col], errors="coerce") 
    # test_df[col] = pd.to_numeric(train_df[col], errors="coerce") 

In [13]:
list_dtype = [train_df[i].dtype for i in train_df.columns]
print(dict((i, list_dtype.count(i)) for i in list_dtype))
# list_dtype = [test_df[i].dtype for i in test_df.columns]
# print(dict((i, list_dtype.count(i)) for i in list_dtype))

{dtype('int64'): 6, dtype('O'): 9}


In [14]:
for col in train_df.columns:
    if 'int' in train_df[col] or 'float' in train_df[col]:
        train_df[col] = train_df[col].fillna(train_df[col].mean())
        # test_df[col] = train_df[col].fillna(test_df[col].mean())

    else:
        mode_val = train_df[col].mode()[0]
        train_df[col] = train_df[col].fillna(mode_val)
        # test_df[col] = test_df[col].fillna(mode_val)

In [15]:
print(train_df.isna().sum().sum())
# print(test_df.isna().sum().sum())

0


In [16]:
print(train_df.shape)
# print(test_df.shape)
# for i in range(len(train_df.columns)):
#         print(train_df.columns[i],train_df[train_df.columns[i]].dtype, test_df.columns[i],test_df[test_df.columns[i]].dtype)

(48842, 15)


In [19]:
print(train_df['income'].value_counts())
# print(test_df[-1].value_counts())

<=50K    37155
>50K     11687
Name: income, dtype: int64


In [21]:
label_encoder = LabelEncoder()

i = 'income'
train_df[i] = label_encoder.fit_transform(train_df[i])
# test_df[i] = label_encoder.transform(test_df[i])

    

In [22]:
print(train_df.shape)
# print(test_df.shape)

(48842, 15)


In [24]:
train_df = pd.get_dummies(train_df)
# test_df = pd.get_dummies(test_df)

In [25]:
print(train_df.shape)
# print(test_df.shape)

(48842, 106)


In [26]:
train_df.head()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,income,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,25,226802,7,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,38,89814,9,0,0,50,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,28,336951,12,0,0,40,1,0,1,0,...,0,0,0,0,0,0,0,1,0,0
3,44,160323,10,7688,0,40,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,18,103497,10,0,0,30,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [30]:
from sklearn.preprocessing import StandardScaler
scl = StandardScaler()
scl.fit(train_df)
scl.transform(train_df)
# scl.transform(test_df)
train_df, test_df = train_test_split(train_df, train_size = 0.9, random_state = 42) 
y_train = train_df['income']
y_test = test_df['income']
X_train = train_df.drop(['income'], axis = 1)
X_test = test_df.drop(['income'], axis = 1)

In [32]:
X_train.head()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
32085,44,207685,9,0,1564,55,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
32448,47,329144,9,4386,0,45,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
47428,22,194829,13,0,0,30,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
23176,21,35424,10,0,0,25,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
32361,27,133937,10,0,0,45,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [33]:
unique, counts = np.unique(y_train, return_counts=True)
print(unique, ":", counts)
unique, counts = np.unique(y_test, return_counts=True)
print(unique, ":", counts)

[0 1] : [30084  9477]
[0 1] : [3358 1038]


In [34]:
train_df.head()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,income,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
32085,44,207685,9,0,1564,55,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32448,47,329144,9,4386,0,45,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
47428,22,194829,13,0,0,30,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
23176,21,35424,10,0,0,25,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
32361,27,133937,10,0,0,45,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [35]:
class CentroidSMOTE():
    def __init__(self, sampling_strategy='auto', random_state=42, k_neighbors=5, m_vertices=3):
        self.sampling_strategy = sampling_strategy
        self.random_state = random_state
        self.k_neighbors = k_neighbors
        self.m_vertices = m_vertices
    
    @staticmethod
    def nearest_neighbor(X, k):
        from sklearn.neighbors import NearestNeighbors
        nbs=NearestNeighbors(n_neighbors=k+1,metric='euclidean',algorithm='kd_tree').fit(X)
        euclidean,indices= nbs.kneighbors(X)
        return indices[:, 1:]
    
    def fit_resample(self, X, y):
        
        (unique, freq) = np.unique(y, return_counts=True)
        frequency = dict(zip(unique, freq))
        max_frequency = max(frequency.values())
        
        if self.sampling_strategy == 'auto':
            sampling_strategy = {}
            for (key, value) in frequency.items():
                sampling_strategy[key] = max_frequency - value
            self.sampling_strategy = sampling_strategy
            
        data = {}
        for (key, value) in self.sampling_strategy.items():
            if value == 0:
                continue
            X_small = X[y == key]
            indices = self.nearest_neighbor(X_small, self.k_neighbors)
            new_data = []
            for idx in np.random.choice(np.arange(len(X_small)), size=value):
                p = X_small[idx]
                nnarray = indices[idx]
                q = X_small[np.random.choice(nnarray, size=self.m_vertices-1, replace=False)]
                new_data.append(np.sum(np.vstack([p, q]), axis=0)/self.m_vertices)
            X_new = np.vstack(new_data)
            data[key] = X_new
        
        for (key, value) in data.items():
            X = np.vstack([X, value])
            y = np.concatenate([y, np.ones(len(value))*key])
        
        X, y = shuffle(X, y, random_state=self.random_state)
        
        return X, y        

In [36]:
def score(X_test, y_test, model):
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    
    print(f"Accuracy: {acc:.6f}")
    print(f"Error Rate: {1-acc}")
    Recall = recall_score(y_test, y_pred, average='macro')      
    print(f"Mean Recall: {Recall}")
    print(f"Balanced Accuracy Score: {balanced_accuracy_score(y_test, y_pred)}")
    Array_prec_recall_f = precision_recall_fscore_support(y_test, y_pred, average='macro')
#                                                                         average = 'micro', 'macro', 'weighted'
    print(f"Precision: {Array_prec_recall_f[0]}")
    print(f"F-Score: {Array_prec_recall_f[2]}")
          
    
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_pred)): 
        if y_test[i]==y_pred[i]==1:
           TP += 1
        if y_pred[i]==1 and y_test[i]!=y_pred[i]:
           FP += 1
        if y_test[i]==y_pred[i]==0:
           TN += 1
        if y_pred[i]==0 and y_test[i]!=y_pred[i]:
           FN += 1

    Selectivity = TN/(TN + FP)
    G_mean = np.sqrt(Selectivity*Recall) 
    print(f"Selectivity: {Selectivity}") 
    print(f"G_mean: {G_mean}")   
    
    
    confusion_matrix = metrics.confusion_matrix(y_test, y_pred)

    cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

    cm_display.plot()
    plt.show()
    
    
    fpr, tpr, thresholds = roc_curve(y_test, y_pred)
    plt.plot(fpr, tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    
   
        

In [37]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model = model.fit(X_train, y_train)
preds = model.predict(X_test)
acc = accuracy_score(preds, y_test)
acc

0.8005004549590536

In [38]:
pipeline = Pipeline([
    ('classifier', KNeighborsClassifier(metric='euclidean'))
])

parameters = {
    'classifier__n_neighbors' : [3, 5, 7, 9, 11],
}

In [39]:
clf = GridSearchCV(pipeline, param_grid = parameters, cv = 5, verbose=True, n_jobs=-1)
best_clf = clf.fit(X_train, y_train)
score(X_test, y_test, best_clf)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


In [ ]:
sm_list = []
from collections import Counter
for i in [3, 5, 7, 9, 11]:
    sm_list.append(SMOTE(random_state=42, k_neighbors=i))
for sm in sm_list:
    X_new, y_new = sm.fit_resample(X_train, y_train)
    print('Resampled dataset shape %s' % Counter(y_new))
    break
    pipeline = Pipeline([
        ('classifier', KNeighborsClassifier())
    ])

    parameters = {
        'classifier__n_neighbors' : [3, 5, 7, 9, 11],
    }

    clf = GridSearchCV(pipeline, param_grid = parameters, cv = 5, verbose=True, n_jobs=-1)
    best_clf = clf.fit(X_new, y_new)
    score(X_test, y_test, best_clf)

In [ ]:
csm_list = []
for (i,j) in [(5,3), (7,3), (9,3), (7,5), (9,7)]:
    csm_list.append(CentroidSMOTE(k_neighbors=i, m_vertices=j))

for csm in csm_list:
    X_new, y_new = csm.fit_resample(X_train, y_train)
    pipeline = Pipeline([
        ('classifier', KNeighborsClassifier())
    ])

    parameters = {
        'classifier__n_neighbors' : [3, 5, 7, 9, 11],
    }

    clf = GridSearchCV(pipeline, param_grid = parameters, cv = 5, verbose=True, n_jobs=-1)
    best_clf = clf.fit(X_new, y_new)
    score(X_test, y_test, best_clf)